<a href="https://colab.research.google.com/github/1900690/gradio/blob/main/gradio_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Google Colab上でwebアプリが作れるGradioの使い方](https://clameyes.com/posts/google-colab-web-app-gradio)

[garadioのplaygroundにサンプルがたくさん](https://www.gradio.app/playground)

##※認証がある場合はID;test pass:gradio

In [1]:
#@title gradioライブラリをインストール
#@markdown この共有リンクの有効期限は1週間です。永続的なホスティングとGPUのアップグレードを無料で行うには、作業ディレクトリのターミナルから `gradio deploy` を実行し、Hugging Face Spaces (https://huggingface.co/spaces) にデプロイしてください。
!pip -q install gradio

In [ ]:
#@title 指定したテキストを返すアプリ
import gradio as gr

# 関数を定義
def greet(name):
    return "Hello " + name + "!"

# Webアプリを作成
app = gr.Interface(fn=greet, inputs="text", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2db3a0419addaceb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title スライダーで数を定義するアプリ
import gradio as gr

# 関数を定義
def count(number):
    return "数字は " + str(number) + "!"

# Webアプリを作成
app = gr.Interface(fn=count, inputs="slider", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d38daa5056d6f920f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Google ColabでGUI｜gradioを利用したウェブアプリの作り方を解説](https://tech.aru-zakki.com/gradio-on-google-colab/)

In [ ]:
#@title 画像をフィルターに通すアプリ
import numpy as np
import gradio as gr

#セピアフィルターを定義
def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

app = gr.Interface(sepia, gr.Image(), "image")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://402e2f7432d4e668b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title　入力した値を覚えるアプリ
import gradio as gr

scores = []

def track_score(score):
    scores.append(score)
    top_scores = sorted(scores, reverse=True)[:3]
    return top_scores

app = gr.Interface(
    track_score,
    gr.Number(label="Score"),
    gr.JSON(label="Top Scores")
)
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd3b70f4d607c1eddb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#アナログメータ針検出アプリをgradio化してみると

In [ ]:
import cv2
import math
import numpy as np
import gradio as gr

def analog(input_img):
        # 指定する中心点（メータの中心）
        cx, cy =(346,304)#横は→に進む、縦は↓に進む
        angle_off_set=-133#pf0の時に０になるように初期値を設定する

        # グレースケール変換 & 二値化
        gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        threshold = 100
        _, img_thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

        # エッジ検出
        edges = cv2.Canny(img_thresh, 50, 200, apertureSize=3)

        # Hough変換で直線を検出（2本の直線を得る）
        minn = None
        for m in range(10, 161, 1):
            lines = cv2.HoughLines(edges, 1, np.pi / 180, m)
            if lines is not None and len(lines) == 2:
                minn = m
                break

        # 直線が2本見つからなかった場合のエラーハンドリング
        if minn is None:
            raise ValueError("直線が2本検出されませんでした。画像のコントラストやエッジ検出のパラメータを調整してください。")

        lines = cv2.HoughLines(edges, 1, np.pi / 180, minn)

        theta_t = []  # 直線の角度
        coefficients = []  # 直線の傾きと切片

        for i in range(len(lines)):
            for rho, theta in lines[i]:
                theta_t.append(theta)

                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho

                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))
                cv2.line(input_img, (x1, y1), (x2, y2), (0, 0, 255), 2)

                # x2-x1が0の場合のエラーハンドリング
                if x2 - x1 == 0:
                    continue  # 無限傾斜を避ける
                a0 = (y2 - y1) / (x2 - x1)
                b0 = y1 - a0 * x1
                coefficients.append((a0, b0))

        # 直線の数をチェック（2本以上ある場合は最初の2本を使用）
        if len(coefficients) < 2:
            raise gr.Error("十分な直線が検出されませんでした。")

        (a1, b1), (a2, b2) = coefficients[:2]  # 最初の2本を選択

        # 交点を計算
        if a1 == a2:
            raise gr.Error("平行な直線のため交点が計算できません。")

        x_t = (b2 - b1) / (a1 - a2)
        y_t = a1 * x_t + b1

        # 交点と中心点を結ぶ線の角度を計算
        angle_rad = np.arctan2(y_t - cy, x_t - cx)
        angle_deg = np.degrees(angle_rad)
        angle =int(angle_deg) +angle_off_set
        angle=angle*-1  #負の角度を修正
        Kpa=8/27*angle
        if Kpa * 10.197 > 0:
          PH=math.log10(Kpa*10.197)
        elif Kpa == 0 :
          PH=0
        else:
          PH=-1
          raise gr.Error("角度がマイナスになりました")

        # 結果を画像に描画
        img_temp=cv2.circle(input_img, (int(cx), int(cy)), 10, (0, 255, 0), -1)  # 中心点を描画
        img_temp=cv2.circle(img_temp, (int(x_t), int(y_t)), 10, (0, 255, 0), -1)  # 交点を描画
        img_temp=cv2.line(img_temp, (cx, cy), (int(x_t), int(y_t)), (255, 0, 0), 2)  # 中心と交点を結ぶ線
        img_temp=cv2.putText(img_temp, f"{PH:.2f} ph", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        return img_temp

app = gr.Interface(
    analog,
    gr.Image(),
    "image",
     examples=[
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/000.jpg"],
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/021.jpg"],
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/026.jpg"]
     ],
    title="アナログメータ針検出アプリ",
    description="アナログメーターの画像をアップロードしてください。画像からから針の角度を検出します。",
    allow_flagging='never'
    )
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d1d3ce4eeec00673aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#開空率測定アプリをgradio化してみると

In [ ]:
import cv2
import numpy as np
import gradio as gr


def detect_sky(input_img, Rmax, Rmin, Gmax, Gmin, Bmax, Bmin):
    # Gradio からの画像は RGB 形式
    src = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)

    # 各色を取り出し
    r = np.array(src[:, :, 2], dtype=np.int16)
    g = np.array(src[:, :, 1], dtype=np.int16)
    b = np.array(src[:, :, 0], dtype=np.int16)

    # 色フィルタリング
    r_mask = np.where((r >= Rmin) & (r <= Rmax), r, 0)
    g_mask = np.where((g >= Gmin) & (g <= Gmax), g, 0)
    b_mask = np.where((b >= Bmin) & (b <= Bmax), b, 0)

    binde_image = np.zeros_like(src)
    binde_image[:, :, 0] = b_mask
    binde_image[:, :, 1] = g_mask
    binde_image[:, :, 2] = r_mask

    # バイナリマスク作成
    binde_image_binary = (r_mask + g_mask + b_mask)
    binde_image_binary = np.where(binde_image_binary > 0, 255, 0).astype("uint8")

    # 輪郭抽出
    contours, _ = cv2.findContours(binde_image_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    img_with_line = np.copy(src)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            img_with_line = cv2.drawContours(img_with_line, contours, i, (0, 0, 255), 1)

    # 可視化用にRGBに変換
    img_with_line_rgb = cv2.cvtColor(img_with_line, cv2.COLOR_BGR2RGB)

    # 面積マスク作成
    img_with_area = np.zeros_like(src)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            img_with_area = cv2.fillPoly(img_with_area, [contours[i][:, 0, :]], (255, 255, 255))

    img_with_area_binary = np.where(np.sum(img_with_area, axis=2) > 0, 255, 0).astype("uint8")
    img_with_area_rgb = np.stack([img_with_area_binary]*3, axis=2)

    # 占有率計算
    all_pixel = img_with_area_binary.size
    sky_pixel = np.sum(img_with_area_binary > 0)
    sky_occupancy_rate = int((sky_pixel / all_pixel) * 100)

    return f"空の面積率: {sky_occupancy_rate}%",img_with_line_rgb, img_with_area_rgb

# Gradioインターフェース
app = gr.Interface(
    fn=detect_sky,
    inputs=[
        gr.Image(type="numpy", label="画像"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="赤上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="赤下限"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="緑上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="緑下限"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="青上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="青下限"),
    ],
    outputs=[
        gr.Label(label="空の面積率"),
        gr.Image(label="空の輪郭表示"),
        gr.Image(label="空の領域マスク")
    ],
    examples=[
    ["https://github.com/1900690/grape-sky-segmentation/releases/download/sample/sample.jpg", 255, 200, 255, 200, 255, 200]
    ],
    allow_flagging='never',
    title="画像から開空率測定アプリ",
    description="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php"
)


app.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://069578a63bc76b58c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#ナスLAIアプリをgradio化してみると

In [20]:
import cv2
import numpy as np
import gradio as gr
import re

# カラーピッカー入力の rgb()/#xxxxxx を BGR タプルに変換する関数
def parse_rgb_string(color_str):
    if color_str.startswith("#"):
        hex_color = color_str.lstrip("#")
        r, g, b = int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)
    else:
        # rgba(r,g,b,a) または rgb(r,g,b)（小数対応）
        match = re.match(r"rgba?\(([\d.]+),\s*([\d.]+),\s*([\d.]+)", color_str)
        if not match:
            raise ValueError(f"色の形式が無効です: {color_str}")
        r, g, b = [int(float(v)) for v in match.groups()]
    return b, g, r  # BGR順

# 空検出メイン関数
def detect_sky(input_img, Rmax, Rmin, Gmax, Gmin, Bmax, Bmin, mask_color_hex):
    src = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)

    r = np.array(src[:, :, 2], dtype=np.int16)
    g = np.array(src[:, :, 1], dtype=np.int16)
    b = np.array(src[:, :, 0], dtype=np.int16)

    r_mask = np.where((r >= Rmin) & (r <= Rmax), r, 0)
    g_mask = np.where((g >= Gmin) & (g <= Gmax), g, 0)
    b_mask = np.where((b >= Bmin) & (b <= Bmax), b, 0)

    binde_image_binary = (r_mask + g_mask + b_mask)
    binde_image_binary = np.where(binde_image_binary > 0, 255, 0).astype("uint8")

    contours, _ = cv2.findContours(binde_image_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    img_with_line = np.copy(src)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            img_with_line = cv2.drawContours(img_with_line, contours, i, (0, 0, 255), 1)

    img_with_line_rgb = cv2.cvtColor(img_with_line, cv2.COLOR_BGR2RGB)

    # カラーピッカーで指定された色でマスク塗りつぶし（葉の領域に色をつけたい場合）
    b_col, g_col, r_col = parse_rgb_string(mask_color_hex)
    # 葉以外領域マスクを作成
    mask_leaf = np.zeros(src.shape[:2], dtype=np.uint8)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            cv2.fillPoly(mask_leaf, [contours[i][:, 0, :]], 255)
    # マスク反転
    mask_inverse = cv2.bitwise_not(mask_leaf)
    img_with_area = np.zeros_like(src)# 元画像サイズでマスク色を塗るキャンバス作成
    img_with_area[mask_inverse == 255] = (b_col, g_col, r_col)  # 葉に色を塗る
    img_with_area_rgb = cv2.cvtColor(img_with_area, cv2.COLOR_BGR2RGB)


    #LAIを算出
    reference_pixel_count=1636516#基準画像のピクセル数
    area_per_pixel_of_the_reference_image=2.170139#基準画像の１ピクセル当たりの面積
    img_with_area_binary = np.where(np.sum(img_with_area, axis=2) > 0, 255, 0).astype("uint8")
    all_pixel = img_with_area_binary.size
    reaf_pixcel = np.sum(img_with_area_binary > 0)
    LAI_rate=(reaf_pixcel*area_per_pixel_of_the_reference_image)/1000000*(all_pixel/reference_pixel_count)

    return f"葉のLAI: {LAI_rate}", img_with_line_rgb, img_with_area_rgb

# Gradio インターフェース
app = gr.Interface(
    fn=detect_sky,
    inputs=[
        gr.Image(type="numpy", label="画像"),
        gr.Slider(0, 255, value=255, step=1, label="赤上限"),
        gr.Slider(0, 255, value=200, step=1, label="赤下限"),
        gr.Slider(0, 255, value=255, step=1, label="緑上限"),
        gr.Slider(0, 255, value=200, step=1, label="緑下限"),
        gr.Slider(0, 255, value=255, step=1, label="青上限"),
        gr.Slider(0, 255, value=200, step=1, label="青下限"),
        gr.ColorPicker(value="#FFFFFF", label="マスクの色（クリックして変更）")
    ],
    outputs=[
        gr.Label(label="葉のLAI"),
        gr.Image(label="葉の輪郭表示"),
        gr.Image(label="葉の領域マスク")
    ],
    examples=[
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/kijun.JPG", 255, 200, 255, 200, 255, 200, "#ff0000"],
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg", 255, 200, 255, 200, 255, 200, "#ff0000"]
    ],
    allow_flagging='never',
    title="ナスLAI測定アプリ",
    description="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php"
)

app.launch()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f93550d95ab6b2d00c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
